In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from tqdm import tqdm
import joblib

from PIL import Image
from io import BytesIO
import requests
from joblib import load

In [2]:
# Гиперпараметры:

train_batch_size = 128
test_batch_size = 256
num_epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device is: {device}")

# Предобучение модели:
pretrained = True

# transforms:
resize = (112, 112)
mean = (0.5, 0.5, 0.5)
std = (0.5, 0.5, 0.5)

# Загрузка датасета и его параметры:
root = 'D:\\online_school\\Project'
train_split = 'train'
test_split = 'test'
download = True

# оптимизатор SGD
learning_rate = 0.001 
momentum = 0.9
weight_decay = 5e-4

Device is: cuda


In [3]:
transform = transforms.Compose([
    transforms.Resize(resize),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

In [4]:
# Загрузка датасета
train_dataset = torchvision.datasets.GTSRB(
    root=root,
    split=train_split,
    transform=transform,
    download=download
)
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=train_batch_size, 
    shuffle=True
)

test_dataset = torchvision.datasets.GTSRB(
    root=root,
    split=test_split,
    transform=transform,
    download=download
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=True
)

In [5]:
# Определение модели
model = models.resnet101(pretrained=pretrained)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 43)  # 101 класс caltech101

C:\Users\Fifil\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Fifil\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
model = model.to(device)

In [7]:
# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=learning_rate,
    momentum=momentum,
    weight_decay=weight_decay
)

In [8]:
# Обучение модели 
for epoch in range (num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch: {epoch+1}, Loss: {running_loss/100:.3f}")

print("Finished Training")

  0%|▍                                                                                 | 1/209 [00:14<51:38, 14.90s/it]


KeyboardInterrupt: 

In [ ]:
# Тестирование модели
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the NN on the test images is: {100 * correct / total} %")

In [ ]:
# Сохранить модель
joblib.dump(model, 'model.pkl')

In [11]:
loaded_model = load('model.pkl')

In [12]:
image_url = 'https://icdn.lenta.ru/images/2019/11/22/08/20191122083838214/original_2fa86a52d3f2a2b09f99c54b19674230.jpg'
image_url

'https://icdn.lenta.ru/images/2019/11/22/08/20191122083838214/original_2fa86a52d3f2a2b09f99c54b19674230.jpg'

In [13]:
# URL изображения
# image_url = 'https://icdn.lenta.ru/images/2019/11/22/08/20191122083838214/original_2fa86a52d3f2a2b09f99c54b19674230.jpg'

# Загрузить изображение
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

image_tensor = transform(image)

# Загрузить модель
loaded_model.eval()

# Переместить тензор изображения и модель на устройство (например, GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_tensor = image_tensor.to(device)
loaded_model = loaded_model.to(device)

# Прогноз
with torch.no_grad():
    output = loaded_model(image_tensor.unsqueeze(0))  # Добавляем batch dimension

# Получить предсказанный класс
predicted_class = output.argmax(dim=1).item()

print(f'Предсказанный класс: {predicted_class}')

ChunkedEncodingError: ('Connection broken: IncompleteRead(3751599 bytes read, 3520923 more expected)', IncompleteRead(3751599 bytes read, 3520923 more expected))